In [1]:
from keras.datasets import cifar10
from keras.models import Model
from keras.layers import Dense, Dropout, SpatialDropout2D, Activation, Flatten
from keras.layers import Input, Convolution2D, Reshape, UpSampling2D
from keras.layers.advanced_activations import LeakyReLU
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l1, l2
from keras.layers.pooling import MaxPooling2D
from keras.optimizers import Adadelta

# from IPython.display import SVG
# from keras.utils.visualize_util import model_to_dot

from keras.utils.visualize_util import plot

Using TensorFlow backend.


In [2]:
disc_optimizer = Adadelta()
GAN_optimizer = Adadelta()

gen_regularizer = l2(1e-4)
disc_regularizer = l2(1e-4)

In [3]:
# Generator Model
g_input = Input(shape=[100])
g_layer = Dense(4*4*512)(g_input)
g_layer = BatchNormalization()(g_layer)
g_layer = Activation('relu')(g_layer)

g_layer = Reshape( [4, 4, 512] )(g_layer)

g_layer = UpSampling2D(size=(2, 2))(g_layer) # 4 -> 8
g_layer = Convolution2D(256, 5, 5, border_mode='same')(g_layer)
g_layer = BatchNormalization(mode=2)(g_layer)
g_layer = Activation('relu')(g_layer)

g_layer = UpSampling2D(size=(2, 2))(g_layer) # 8 -> 16
g_layer = Convolution2D(128, 5, 5, border_mode='same')(g_layer)
g_layer = BatchNormalization(mode=2)(g_layer)
g_layer = Activation('relu')(g_layer)

g_layer = UpSampling2D(size=(2, 2))(g_layer) # 16 -> 32
g_layer = Convolution2D(3, 5, 5, border_mode='same', W_regularizer=gen_regularizer)(g_layer)
g_output = Activation('sigmoid')(g_layer)

generator = Model(g_input, g_output)
generator.compile(loss='mean_squared_error', optimizer=GAN_optimizer)

In [4]:
# SVG(model_to_dot(generator).create(prog='dot', format='svg'))
plot(generator, to_file='generator.png', show_shapes='True')

In [5]:
d_input = Input(shape=(32, 32, 3))

d_layer = Convolution2D(32, 3, 3, border_mode='same', W_regularizer=disc_regularizer)(d_input)
d_layer = LeakyReLU()(d_layer)

d_layer = Convolution2D(32, 3, 3, border_mode='same')(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = MaxPooling2D(pool_size=(2, 2))(d_layer) # 32 -> 16
d_layer = SpatialDropout2D(0.25)(d_layer)

d_layer = Convolution2D(64, 3, 3, border_mode='same', W_regularizer=disc_regularizer)(d_layer)
d_layer = LeakyReLU()(d_layer)

d_layer = Convolution2D(64, 3, 3, border_mode='same')(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = MaxPooling2D(pool_size=(2, 2))(d_layer) # 16 -> 8
d_layer = SpatialDropout2D(0.25)(d_layer)

d_layer = Convolution2D(96, 3, 3, border_mode='same', W_regularizer=disc_regularizer)(d_layer)
d_layer = LeakyReLU()(d_layer)

d_layer = Convolution2D(96, 3, 3, border_mode='same')(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = MaxPooling2D(pool_size=(2, 2))(d_layer) # 8 -> 4
d_layer = SpatialDropout2D(0.25)(d_layer)

d_layer = Convolution2D(128, 3, 3, border_mode='same', W_regularizer=disc_regularizer)(d_layer)
d_layer = LeakyReLU()(d_layer)

d_layer = Convolution2D(128, 3, 3, border_mode='same')(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = MaxPooling2D(pool_size=(2, 2))(d_layer) # 4 -> 2
d_layer = SpatialDropout2D(0.25)(d_layer)

d_layer = Flatten()(d_layer) # 2 x 2 x 128 -> 512

d_layer = Dense(256)(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = Dropout(0.5)(d_layer)

d_layer = Dense(256)(d_layer)
d_layer = LeakyReLU()(d_layer)
d_layer = Dropout(0.5)(d_layer)

d_layer = Dense(3)(d_layer)
d_output = Activation('softmax')(d_layer)

discriminator = Model(d_input, d_output)
discriminator.compile(loss='categorical_crossentropy', optimizer=disc_optimizer)

In [6]:
# SVG(model_to_dot(discriminator).create(prog='dot', format='svg'))
plot(discriminator, to_file='discriminator.png', show_shapes='True')

In [7]:
# GAN Model
gan_input = Input(shape=[100])
gan_layer = generator(gan_input)
gan_output = discriminator(gan_layer)
GAN = Model(gan_input, gan_output)
GAN.compile(loss='categorical_crossentropy', optimizer=GAN_optimizer)

In [8]:
# SVG(model_to_dot(GAN).create(prog='dot', format='svg'))
plot(GAN, to_file='GAN.png', show_shapes='True')